In [111]:
from dataclasses import dataclass
import datetime
from germansentiment import SentimentModel
import html
import pandas as pd 
import re
import snscrape.modules.twitter as sntwitter
import sqlite3
import torch

In [99]:
pd.set_option('display.max_colwidth', None)
db_file = 'tweets-de.db'

In [102]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)

In [119]:
@dataclass
class Query:
    search_term: str
    lang: str
    date_from: datetime.datetime
    date_to: datetime.datetime

    def __str__(self):
        return(self.search_term  + " lang:" + self.lang + " until:" + self.date_to.strftime('%Y-%m-%d') + " since:" + self.date_from.strftime('%Y-%m-%d'))

In [257]:
def preprocess_tweet(tweet):
    tweet.rawContent = html.unescape(tweet.rawContent)
    tweet.rawContent = re.sub("@[A-Za-z0-9_]+", "", tweet.rawContent).strip()
    tweet.rawContent = re.sub("#", "", tweet.rawContent).strip()
    tweet.rawContent = re.sub(r"http\S+", "", tweet.rawContent)
    tweet.rawContent = re.sub("(\\n)+", " ", tweet.rawContent)
    tweet.rawContent = re.sub("(\\'er)", "er", tweet.rawContent)
    if tweet.hashtags is not None:
        tweet.hashtags = ";".join(tweet.hashtags)
    return tweet

In [262]:
def gather_tweets_to_sql(query, conn, limit = 100):
    sql_insert = ''' INSERT INTO tweets(id, date, user, lang, search_term, content, hashtags, sentiment)
              VALUES(?, ?, ?, ?, ?, ?, ?, ?) '''
    i = 0
    for tweet in sntwitter.TwitterSearchScraper(str(query)).get_items():
        if i == limit:
            break
        else:
            i += 1
            tweet = preprocess_tweet(tweet)
            sql_tweet= (tweet.id, tweet.date, tweet.user.username, tweet.lang, query.search_term, tweet.rawContent, tweet.hashtags, None)
            cur = conn.cursor()
            cur.execute(sql_insert, sql_tweet)
            conn.commit()

In [235]:
sql_create_tweets_table = """CREATE TABLE tweets (
                                       id integer NOT NULL,
                                       date TIMESTAMP NOT NULL,
                                       user TEXT NOT NULL,
                                       lang TEXT NOT NULL,
                                       search_term TEXT NOT NULL,
                                       content TEXT NOT NULL,
                                       hashtags TEXT,
                                       sentiment integer,
                                       PRIMARY KEY (id));"""

In [260]:
# create a database connection
conn = create_connection(db_file)
create_table(conn, sql_create_tweets_table)
conn.close

<function Connection.close()>

In [265]:
q = Query("Bremen", "de", datetime.datetime(2022, 12, 30), datetime.datetime(2023, 1, 2))

In [264]:
conn = create_connection(db_file)
gather_tweets_to_sql(q, conn, 5000)

In [83]:
query = "#berlin lang:de until:2023-01-02 since:2022-12-31"
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.user.username, tweet.rawContent])
        
df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
print(df)

                          Date             User  \
0    2023-01-01 23:56:38+00:00      SamJimmyjoe   
1    2023-01-01 23:51:47+00:00          _AIQIA_   
2    2023-01-01 23:51:31+00:00  WetterPrenzBerg   
3    2023-01-01 23:48:19+00:00          Wolf_X5   
4    2023-01-01 23:46:12+00:00      RDR2_Zocker   
...                        ...              ...   
2151 2022-12-31 00:40:29+00:00       GrafZahl13   
2152 2022-12-31 00:14:06+00:00       nafrodamus   
2153 2022-12-31 00:11:57+00:00      Wolf__Odins   
2154 2022-12-31 00:04:54+00:00      Wolf__Odins   
2155 2022-12-31 00:01:04+00:00        UweHiksch   

                                                                                                                                                                                                                                                                                                                                         Tweet  
0                                                 

In [95]:
vars(tweet)

{'url': 'https://twitter.com/UweHiksch/status/1608976494621073409',
 'date': datetime.datetime(2022, 12, 31, 0, 1, 4, tzinfo=datetime.timezone.utc),
 'rawContent': 'Im Bezirk regt sich Widerstand gegen AfD-Parteizentrale in Wittenau \n\nhttps://t.co/Ngd0xCYXpe \n\n#antifa #antira #noAfD #AfDstoppen #StoppAfD #Berlin',
 'renderedContent': 'Im Bezirk regt sich Widerstand gegen AfD-Parteizentrale in Wittenau \n\nleute.tagesspiegel.de/reinickendorf/… \n\n#antifa #antira #noAfD #AfDstoppen #StoppAfD #Berlin',
 'id': 1608976494621073409,
 'user': User(username='UweHiksch', id=22991930, displayname='Uwe Hiksch', rawDescription='Seit vielen Jahren aktiv in Anti-Atom-Bewegung, Antifa, Friedensbewegung, NaturFreunden und Umweltbewegung.', renderedDescription='Seit vielen Jahren aktiv in Anti-Atom-Bewegung, Antifa, Friedensbewegung, NaturFreunden und Umweltbewegung.', descriptionLinks=None, verified=False, created=datetime.datetime(2009, 3, 5, 22, 42, 51, tzinfo=datetime.timezone.utc), followersC

In [84]:
df['Tweet'] = df.apply(lambda row: preprocess_tweet(row['Tweet'] ), axis = 1)

In [72]:
df.iloc[25:40,:]

,Date,User,Tweet
25,2023-01-01 23:08:21+00:00,nologyy,"Muslimische Migranten: Wow, Deutschland ist so rassistisch, warum sind wir unbeliebter als ukrainische Migranten? Auch muslimische Migranten: Yeah, F*ck the Police, F*ck Deutschland, wir scheißen auf eure Regeln und legen eure Stadt in Schutt und Asche. Berlin"
26,2023-01-01 23:05:20+00:00,Ichsagsja,"Es ist entsetzlich, was da in Berlin in der letzten Nacht abgegangen ist. Ich bin mir allerdings nicht sicher, ob die deutsche Gesellschaft in der Lage ist, die Ursachen klar zu analysieren. Ein unideologischer Blick auf Geschlecht, Schicht, kulturellem Hintergrund,... 1/"
27,2023-01-01 23:03:09+00:00,GreenKraut,"Erschreckend Bilder von Angriffen auf Polizei, Feuerwehr und Hilfskräfte mit Böllern in der Berlin-er Silversternacht Eine Einschränkung von Feuerwerk in den Händen von Privatleuten wäre somit logisch und wünschenswert. Böllerverbot via ChangeOrg"
28,2023-01-01 23:02:44+00:00,TributeCopCar,⚫️🔵⚫️ 🇪🇺🇩🇪 Berlin Jahreswechsel Dem ist nichts hinzuzufügen:
29,2023-01-01 22:59:57+00:00,stfnkng,In Berlin zeigt sich wie die 🇩🇪Staatlichkeit zerfällt und scheitert. Ein Staat der das Gewaltmonopol zunehmend nicht mehr zum Schutze der Bevölkerung aufrechterhalten kann wird zurecht als „failing State“ bezeichnet.
30,2023-01-01 22:59:47+00:00,hintz,Es ist Krieg in Europa. Amateur Pyromanen Berlin Silvester Neujahr Straftat Lärm giftig gefährlich Feuer brennen Angriff kriminell sparen Zerstörung Wut
31,2023-01-01 22:59:05+00:00,TributeCopCar,⚫️🔵⚫️ 🇪🇺🇩🇪 Berlin Das DARF so NICHT weitergehen!! 👉🏼 WeBackTheBlue !
32,2023-01-01 22:58:31+00:00,Migista01,"Huhu Frau , wenn Sie bei einer Razzia mal mehr als 10 illegale Waffen, wie bei den ""Reichsbürgern"", entdecken wollen, hätten Sie nur unter den westasiatischen Böller-Kriminellen in Berlin suchen müssen. Dafür hätten weniger als 3.000 Polizisten gereicht.🤷🤔🙈🙉🙊"
33,2023-01-01 22:54:15+00:00,BraasGisela,"Kurz gesagt, ist Berlin ein Problem und zwar in JEDER HINSICHT!! boellerverbot"
34,2023-01-01 22:54:07+00:00,vonAVENdeForum,Was für ein asoziales Dreckspack hat in Berlin Einsatzkräfte angegriffen u in einen Hinterhalt gelockt u mit Waffen bedroht u angegriffen? Ihr seid doch nicht mehr normal! IHR seid die Verbrecher nicht die Klimaaktivisten! Kriminell Berlinsilvester DankeFeuerwehrBerlin


In [93]:
model = SentimentModel()
df['Sentiment'] = model.predict_sentiment(df.iloc[:,2])
df['Sentiment'].value_counts() 

neutral     1585
negative     507
positive      64
Name: Sentiment, dtype: int64

In [125]:
model = SentimentModel()

In [126]:
model

In [137]:
df.iloc[38, 2]

'Silvester2022 hat mal wieder gezeigt, dass Berlin ein Failed State ist. Da wird einmal mehr das Ergebnis dunkelrot-rot-grüner Politik deutlich.'

In [138]:
df.iloc[38, 3]

'neutral'

In [139]:
model.predict_sentiment(["Hallo was ist los?"])

['negative']

In [124]:
del model
torch.cuda.empty_cache()